In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
import re

In [2]:
train = pd.read_csv('../1001_proj/train.csv/train.csv')
test = pd.read_csv('../1001_proj/test.csv/test.csv')
song = pd.read_csv('../1001_proj/songs.csv/songs.csv')
members =  pd.read_csv('../1001_proj/members.csv/members.csv')

In [3]:
train = train.merge(song,on= 'song_id',how = 'left')
test = test.merge(song,on = 'song_id',how = 'left')

In [4]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train)

In [11]:
tuf = train.groupby('msno')['target'].agg(['sum','count']) #Laplacian smoothing
tuf = tuf.reset_index()
tuf.columns.values[1:3] = ['user_sum','user_count']
train = train.merge(tuf,on = 'msno',how = 'left')
train['user_freq'] = (train.user_sum-train.target+1)/(train.user_count+1)
train['user_count'] = train['user_count'] - 1
train = train.drop('user_sum',1)

In [12]:
tuf['user_freq'] = (tuf.user_sum+1)/(tuf.user_count+2)
valid = valid.merge(tuf,on = 'msno',how = 'left')
valid.loc[pd.isnull(valid.user_count),'user_count'] = 0
valid.loc[pd.isnull(valid.user_freq),'user_freq'] = 0.5
valid = valid.drop('user_sum',1)

In [13]:
tuf['user_freq'] = (tuf.user_sum+1)/(tuf.user_count+2)
test = test.merge(tuf,on = 'msno',how = 'left')
test.loc[pd.isnull(test.user_count),'user_count'] = 0
test.loc[pd.isnull(test.user_freq),'user_freq'] = 0.5
test = test.drop('user_sum',1)

In [14]:
tuf = train.groupby('song_id')['target'].agg(['sum','count']) #Laplacian smoothing
tuf = tuf.reset_index()
tuf.columns.values[1:3] = ['song_sum','song_count']
train = train.merge(tuf,on = 'song_id',how = 'left')
train['song_freq'] = (train.song_sum-train.target+1)/(train.song_count+1)
train['song_count'] = train['song_count'] -1
train = train.drop('song_sum',1)

In [15]:
tuf['song_freq'] = (tuf.song_sum+1)/(tuf.song_count+2)
valid = valid.merge(tuf,on = 'song_id',how = 'left')
valid.loc[pd.isnull(valid.song_count),'song_count'] = 0
valid.loc[pd.isnull(valid.song_freq),'song_freq'] = 0.5
valid = valid.drop('song_sum',1)

In [16]:
tuf['song_freq'] = (tuf.song_sum+1)/(tuf.song_count+2)
test = test.merge(tuf,on = 'song_id',how = 'left')
test.loc[pd.isnull(test.song_count),'song_count'] = 0
test.loc[pd.isnull(test.song_freq),'song_freq'] = 0.5
test = test.drop('song_sum',1)

In [17]:
def split(x):
    l = re.split("[+|&/]",x)
    l = [i.strip() for i in list(l)]
    return l

In [18]:
train.artist_name = train.artist_name.fillna("no_rec")
valid.artist_name = valid.artist_name.fillna("no_rec")
test.artist_name = test.artist_name.fillna("no_rec")

In [19]:
train_art = set()
for name in train.artist_name.unique():
    train_art.add(name)
    for sub in split(name):
        train_art.add(sub.strip())

In [20]:
valid_art_diff = set()
for name in valid.artist_name.unique():
    indic = [sub.strip() in train_art for sub in split(name)]
    if not any(indic):
        valid_art_diff.add(name)

In [21]:
test_art_diff = set()
for name in test.artist_name.unique():
    indic = [sub.strip() in train_art for sub in split(name)]
    if not any(indic):
        test_art_diff.add(name)

In [22]:
valid_tt_csame = set()
for name in valid.artist_name.unique():
     if len(split(name)) > 1 and name in train_art:
            valid_tt_csame.add(name)

In [23]:
tt_csame = set()
for name in test.artist_name.unique():
     if len(split(name)) > 1 and name in train_art:
            tt_csame.add(name)

In [24]:
tt_single = set()
for name in train.artist_name.unique():
    if len(split(name)) == 1:
        tt_single.add(name)

In [25]:
tt_cdump = set()
for name in train.artist_name.unique():
    if len(split(name)) > 1 and (not name in tt_csame or not name in valid_tt_csame):
        tt_cdump.add(name)

In [26]:
artdict = dict(zip(list(train_art),range(len(train_art))))
userdict = dict(zip(train.msno.unique(),range(train.msno.nunique())))
v = train.groupby(['msno','artist_name'])['target'].agg(['sum','count'])
v = v.reset_index()
v.columns.values[2:] = ['uasum','uacount']
uamat = sparse.lil_matrix((len(userdict),2*len(artdict)))
v['mind'] = v.msno.apply(lambda x: userdict[x])
v['amind'] = v.artist_name.apply(lambda x: artdict[x])
for i in v.index:
    uamat[v.mind[i],2*v.amind[i]] = v.uasum[i]
    uamat[v.mind[i],2*v.amind[i]+1] = v.uacount[i]

In [27]:
disdict = {}
for key, ind in artdict.items():
    l = [artdict[sub] for sub in split(key) if not sub in tt_single]
    if len(l) > 0:
        disdict[key] = (l,ind)
uamat = sparse.csc_matrix(uamat)
for key, ind in disdict.items():
    for sub in ind[0]:
        uamat[:,2*sub] += uamat[:,2*ind[1]]
        uamat[:,2*sub+1] += uamat[:,2*ind[1]+1]

/Users/Dylan/anaconda/envs/3point6/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [28]:
oart = train.groupby('artist_name')['target'].agg(['sum','count'])
oart = oart.reset_index()
oart.columns.values[1:3] = ['art_sum','art_count']
train = train.merge(oart,on='artist_name',how = 'left')
train['art_freq'] = (train.art_sum-train.target+1)/(train.art_count+1)
traindump = train[train.artist_name.isin(list(tt_cdump))]
train['is_art_dump'] = 0
uasum = uamat.sum(axis = 0)
for name,group in traindump.groupby("artist_name"):
    subind = np.array([artdict[sub.strip()] for sub in split(name)])
    gasum = uasum[0,2*subind].sum()/len(subind)
    gacount = uasum[0,2*subind+1].sum()/len(subind)
    for ind in group.index:
        train.set_value(ind,'art_sum',gasum)
        train.set_value(ind,'art_count',gacount)
        train.set_value(ind,'art_freq',gasum/gacount)
        train.set_value(ind,'is_art_dump',1)
train = train.drop('art_sum',1)
train.art_count = train.art_count -1

In [29]:
oart['art_freq'] = oart.art_sum/oart.art_count
valid = valid.merge(oart,on='artist_name',how = 'left')
valid['is_art_dump'] = 0
validnan = valid[pd.isnull(valid.art_sum)]
for name, group in validnan.groupby("artist_name"):
    subind = np.array([artdict[sub.strip()] for sub in split(name) if sub.strip() in train_art])
    if subind.size == 0:
        for ind in group.index:
            valid.set_value(ind,'art_count',0)
            valid.set_value(ind,'art_freq',0.5)
    else:
        gasum = uasum[0,2*subind].sum()/len(subind)
        gacount = uasum[0,2*subind+1].sum()/len(subind)
        for ind in group.index:
            valid.set_value(ind,'art_count',gacount)
            valid.set_value(ind,'art_freq',gasum/gacount)
            valid.set_value(ind,'is_art_dump',1)
valid = valid.drop('art_sum',1)

In [30]:
test = test.merge(oart,on='artist_name',how = 'left')
test['is_art_dump'] = 0
testnan = test[pd.isnull(test.art_sum)]
for name, group in testnan.groupby("artist_name"):
    subind = np.array([artdict[sub.strip()] for sub in split(name) if sub.strip() in train_art])
    if subind.size == 0:
        for ind in group.index:
            test.set_value(ind,'art_count',0)
            test.set_value(ind,'art_freq',0.5)
    else:
        gasum = uasum[0,2*subind].sum()/len(subind)
        gacount = uasum[0,2*subind+1].sum()/len(subind)
        for ind in group.index:
            test.set_value(ind,'art_count',gacount)
            test.set_value(ind,'art_freq',gasum/gacount)
            test.set_value(ind,'is_art_dump',1)
test = test.drop('art_sum',1)

In [31]:
train = train.merge(v,on = ['msno','artist_name'],how = 'left')
train['uafreq'] = (train['uasum'] - train['target']+1)/(train['uacount']+1)
train['uacount'] = train['uacount'] -1
for name,group in train[train.artist_name.isin(list(tt_cdump))].groupby('artist_name'):
    subcolin = np.array([artdict[sub.strip()] for sub in split(name)])
    vsum = uamat[:,2*subcolin][group.mind,:].sum(axis =1)
    vcount = uamat[:,2*subcolin+1][group.mind,:].sum(axis = 1)
    gi = group.index ; gs = group.shape[0]
    for i in range(gs):
        train.set_value(gi[i],'uafreq',vsum[i]/vcount[i] if vcount[i] > 0 else 0.5)
        train.set_value(gi[i],'uacount',vcount[i]/len(subcolin))
train = train.drop(['mind','amind','uasum'],1)

In [32]:
trainmsno = train.msno.unique()
v['uafreq'] = v.uasum/v.uacount
valid = valid.merge(v,on = ['msno','artist_name'],how = 'left')
validnan = valid[pd.isnull(valid.uacount)]
validnu = validnan[~validnan.msno.isin(trainmsno)]
for name,group in validnu.groupby(['msno','artist_name']):
    subcolin = np.array([artdict[sub.strip()] for sub in split(name[1]) if sub.strip() in train_art])
    indic = len(subcolin) > 0 
    overallfreq = uasum[0,2*subcolin].sum()/uasum[0,2*subcolin+1].sum() if indic else 0.5
    for ind in group.index:
        valid.set_value(ind,'uacount',0)
        valid.set_value(ind,'uafreq',overallfreq)
validrem = valid[pd.isnull(valid.uafreq)]
validrem['mind']= validrem.msno.apply(lambda x: userdict[x])
for name,group in validrem.groupby('artist_name'):
    subcolin = np.array([artdict[sub.strip()] for sub in split(name) if sub.strip() in train_art])
    if len(subcolin) > 0:
        vsum = uamat[:,2*subcolin][group.mind,:].sum(axis = 1)
        vcount = uamat[:,2*subcolin+1][group.mind,:].sum(axis =1)
        gi = group.index
        for i in range(group.shape[0]):
            valid.set_value(gi[i],'uafreq',vsum[i]/vcount[i] if vcount[i] > 0 else 0.5)
            valid.set_value(gi[i],'uacount',vcount[i]/len(subcolin))
    else:
        for ind in group.index:
            valid.set_value(ind,'uacount',0)
            valid.set_value(ind,'uafreq',group.user_freq[ind])
valid = valid.drop(['mind','amind','uasum'],1)

/Users/Dylan/anaconda/envs/3point6/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [33]:
test = test.merge(v,on = ['msno','artist_name'],how = 'left')
testnan = test[pd.isnull(test.uacount)]
testnu = testnan[~testnan.msno.isin(trainmsno)]
for name,group in testnu.groupby(['msno','artist_name']):
    subcolin = np.array([artdict[sub.strip()] for sub in split(name[1]) if sub.strip() in train_art])
    indic = len(subcolin) > 0 
    overallfreq = uasum[0,2*subcolin].sum()/uasum[0,2*subcolin+1].sum() if indic else 0.5
    for ind in group.index:
        test.set_value(ind,'uacount',0)
        test.set_value(ind,'uafreq',overallfreq)
testrem = test[pd.isnull(test.uafreq)]
testrem['mind']= testrem.msno.apply(lambda x: userdict[x])
for name,group in testrem.groupby('artist_name'):
    subcolin = np.array([artdict[sub.strip()] for sub in split(name) if sub.strip() in train_art])
    if len(subcolin) > 0:
        vsum = uamat[:,2*subcolin][group.mind,:].sum(axis = 1)
        vcount = uamat[:,2*subcolin+1][group.mind,:].sum(axis =1)
        gi = group.index
        for i in range(group.shape[0]):
            test.set_value(gi[i],'uafreq',vsum[i]/vcount[i] if vcount[i] > 0 else 0.5)
            test.set_value(gi[i],'uacount',vcount[i]/len(subcolin))
    else:
        for ind in group.index:
            test.set_value(ind,'uacount',0)
            test.set_value(ind,'uafreq',group.user_freq[ind])
test = test.drop(['mind','amind','uasum'],1)

/Users/Dylan/anaconda/envs/3point6/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [35]:
train.genre_ids = train.genre_ids.fillna('-1')
test.genre_ids = test.genre_ids.fillna('-1')
valid.genre_ids = valid.genre_ids.fillna('-1')

In [36]:
train_genre = set()
for genre in train.genre_ids:
    train_genre.add(genre)
    if not pd.isnull(genre) and "|" in genre:
        for sub in genre.split("|"):
            train_genre.add(sub)

In [37]:
valid_genre_diff = set()  #只有五个单个genre没有
for name in valid.genre_ids.unique():
    if not pd.isnull(name) and not "|" in name and not name in train_genre:
        valid_genre_diff.add(name)
    if not pd.isnull(name) and "|" in name:
        indic = [x in train_genre for x in name.split("|")]
        if not any(indic):
            valid_genre_diff.add(name)

In [38]:
test_genre_diff = set()  #只有五个单个genre没有
for name in test.genre_ids.unique():
    if not pd.isnull(name) and not "|" in name and not name in train_genre:
        test_genre_diff.add(name)
    if not pd.isnull(name) and "|" in name:
        indic = [x in train_genre for x in name.split("|")]
        if not any(indic):
            test_genre_diff.add(name)

In [42]:
train_gsame = set()
for genre in (list(test.genre_ids.unique()) + list(valid.genre_ids.unique())):
    if not pd.isnull(genre) and "|" in genre and genre in train_genre:
        train_gsame.add(genre)

In [43]:
train_gdump = set()
for genre in train.genre_ids.unique():
    if not pd.isnull(genre) and "|" in genre and not genre in train_gsame:
        train_gdump.add(genre)

In [44]:
train_gsingle = [x for x in train.genre_ids.unique() if not pd.isnull(x) and not "|" in x]


In [45]:
valid_newsingle = set()
for x in valid.genre_ids.unique():
    if not pd.isnull(x) and "|" in x:
        indc = [sub for sub in x.split("|") if sub in train_gsingle]
        if len(indc) == 0:
            valid_newsingle.add(sub)

In [46]:
test_newsingle = set()
for x in test.genre_ids.unique():
    if not pd.isnull(x) and "|" in x:
        indc = [sub for sub in x.split("|") if sub in train_gsingle]
        if len(indc) == 0:
            test_newsingle.add(sub)

In [47]:
valid_compnew = set()
for genre in valid.genre_ids.unique():
    if not pd.isnull(genre) and "|" in genre and not genre in train_genre:
        for sub in genre.split("|"):
            if not sub in train_gsingle and sub in train_genre:
                valid_compnew.add(genre) 

In [48]:
test_compnew = set()
for genre in test.genre_ids.unique():
    if not pd.isnull(genre) and "|" in genre and not genre in train_genre:
        for sub in genre.split("|"):
            if not sub in train_gsingle and sub in train_genre:
                test_compnew.add(genre) 

In [49]:
genredict = dict(zip(list(train_genre),range(len(train_genre))))
ugmat = sparse.lil_matrix((len(userdict),2*len(genredict)))
v = train.groupby(['msno','genre_ids'])['target'].agg(['sum','count'])
v = v.reset_index()
v.columns.values[2:] = ['ugsum','ugcount']
v['mind'] = v.msno.apply(lambda x: userdict[x])
v['gmind'] = v.genre_ids.apply(lambda x: genredict[x])
for i in v.index:
    ugmat[v.mind[i],2*v.gmind[i]] = v.ugsum[i]
    ugmat[v.mind[i],2*v.gmind[i]+1] = v.ugcount[i]
ugmat = sparse.csc_matrix(ugmat)
for dump in train.genre_ids.unique():
    subp = [dumpart for dumpart in dump.split("|") if dumpart not in train_gsingle]
    for sub in subp:
        ugmat[:,2*genredict[sub]] += ugmat[:,2*genredict[dump]]
        ugmat[:,2*genredict[sub]+1] += ugmat[:,2*genredict[dump]+1]

/Users/Dylan/anaconda/envs/3point6/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [50]:
ogenre = train.groupby('genre_ids')['target'].agg(['sum','count'])
ogenre = ogenre.reset_index()
ogenre.columns.values[1:3] = ['genre_sum','genre_count']
train.genre_ids = train.genre_ids.astype('str')
ogenre.genre_ids = ogenre.genre_ids.astype('str')
train = train.merge(ogenre,on='genre_ids',how = 'left')
train['genre_freq'] = (train.genre_sum-train.target+1)/(train.genre_count+1)
traindump = train[train.genre_ids.isin(list(train_gdump))]
train['is_genre_dump'] = 0
ugsum = ugmat.sum(axis = 0)
for name,group in traindump.groupby("genre_ids"):
    subind = np.array([genredict[sub] for sub in name.split("|")])
    gasum = ugsum[0,2*subind].sum()/len(subind)
    gacount = ugsum[0,2*subind+1].sum()/len(subind)
    for ind in group.index:
        train.set_value(ind,'genre_sum',gasum)
        train.set_value(ind,'genre_count',gacount)
        train.set_value(ind,'genre_freq',gasum/gacount)
        train.set_value(ind,'is_genre_dump',1)
train = train.drop('genre_sum',1)
train.genre_count = train.genre_count -1 

In [51]:
ogenre['genre_freq'] = ogenre.genre_sum/ogenre.genre_count
valid = valid.merge(ogenre,on='genre_ids',how = 'left')
valid['is_genre_dump'] = 0
validnan = valid[pd.isnull(valid.genre_sum)]
for name, group in validnan.groupby("genre_ids"):
    if not "|" in name:
        for ind in group.index:
            valid.set_value(ind,'genre_count',0)
            valid.set_value(ind,'genre_freq',0.5)
    else:
        subind = np.array([genredict[sub] for sub in name.split("|") if sub in train_genre])
        gasum = ugsum[0,2*subind].sum()/len(subind)
        gacount = ugsum[0,2*subind+1].sum()/len(subind)
        for ind in group.index:
            valid.set_value(ind,'genre_count',gacount)
            valid.set_value(ind,'genre_freq',gasum/gacount)
            valid.set_value(ind,'is_genre_dump',1)
valid = valid.drop('genre_sum',1)

In [52]:
ogenre['genre_freq'] = ogenre.genre_sum/ogenre.genre_count
test = test.merge(ogenre,on='genre_ids',how = 'left')
test['is_genre_dump'] = 0
testnan = test[pd.isnull(test.genre_sum)]
for name, group in testnan.groupby("genre_ids"):
    if not "|" in name:
        for ind in group.index:
            test.set_value(ind,'genre_count',0)
            test.set_value(ind,'genre_freq',0.5)
    else:
        subind = np.array([genredict[sub] for sub in name.split("|") if sub in train_genre])
        gasum = ugsum[0,2*subind].sum()/len(subind)
        gacount = ugsum[0,2*subind+1].sum()/len(subind)
        for ind in group.index:
            test.set_value(ind,'genre_count',gacount)
            test.set_value(ind,'genre_freq',gasum/gacount)
            test.set_value(ind,'is_genre_dump',1)
test = test.drop('genre_sum',1)

In [53]:
train = train.merge(v,on = ['msno','genre_ids'],how = 'left')
train['ugfreq'] = (train['ugsum'] - train['target']+1)/(train['ugcount']+1)
train['ugcount'] = train['ugcount'] -1
for name,group in train[train.genre_ids.isin(list(train_gdump))].groupby('genre_ids'):
    subcolin = np.array([genredict[sub.strip()] for sub in name.split("|")])
    vsum = ugmat[:,2*subcolin][group.mind,:].sum(axis =1)
    vcount = ugmat[:,2*subcolin+1][group.mind,:].sum(axis = 1)
    gi = group.index ; gs = group.shape[0]
    for i in range(gs):
        train.set_value(gi[i],'ugfreq',vsum[i]/vcount[i] if vcount[i] > 0 else 0.5)
        train.set_value(gi[i],'ugcount',vcount[i]/len(subcolin))
train = train.drop(['mind','gmind','ugsum'],1)

In [54]:
trainmsno = train.msno.unique()
v['ugfreq'] = v.ugsum/v.ugcount
valid = valid.merge(v,on = ['msno','genre_ids'],how = 'left')
validnan = valid[pd.isnull(valid.ugcount)]
validnu = validnan[~validnan.msno.isin(trainmsno)]
for name,group in validnu.groupby(['msno','genre_ids']):
    subcolin = np.array([genredict[sub.strip()] for sub in name[1].split("|") if sub in train_genre])
    overallfreq = ugsum[0,2*subcolin].sum()/ugsum[0,2*subcolin+1].sum()
    indic = len(subcolin) > 0
    for ind in group.index:
        valid.set_value(ind,'ugcount',0)
        valid.set_value(ind,'ugfreq',overallfreq if indic else 0.5)
validrem = valid[pd.isnull(valid.ugfreq)]
validrem['mind']= validrem.msno.apply(lambda x: userdict[x])
for name,group in validrem.groupby('genre_ids'):
    subcolin = np.array([genredict[sub.strip()] for sub in name.split("|") if sub in train_genre])
    if len(subcolin) > 0:
        vsum = ugmat[:,2*subcolin][group.mind,:].sum(axis = 1)
        vcount = ugmat[:,2*subcolin+1][group.mind,:].sum(axis =1)
        gi = group.index
        for i in range(group.shape[0]):
            valid.set_value(gi[i],'ugfreq',vsum[i]/vcount[i] if vcount[i] > 0 else 0.5)
            valid.set_value(gi[i],'ugcount',vcount[i]/len(subcolin))
    else:
        for ind in group.index:
            valid.set_value(ind,'ugcount',0)
            valid.set_value(ind,'ugfreq',group.user_freq[ind])
valid = valid.drop(['mind','gmind','ugsum'],1)

/Users/Dylan/anaconda/envs/3point6/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [55]:
test = test.merge(v,on = ['msno','genre_ids'],how = 'left')
testnan = test[pd.isnull(test.ugcount)]
testnu = testnan[~testnan.msno.isin(trainmsno)]
for name,group in testnu.groupby(['msno','genre_ids']):
    subcolin = np.array([genredict[sub.strip()] for sub in name[1].split("|") if sub in train_genre])
    overallfreq = ugsum[0,2*subcolin].sum()/ugsum[0,2*subcolin+1].sum()
    indic = len(subcolin) > 0
    for ind in group.index:
        test.set_value(ind,'ugcount',0)
        test.set_value(ind,'ugfreq',overallfreq if indic else 0.5)
testrem = test[pd.isnull(test.ugfreq)]
testrem['mind']= testrem.msno.apply(lambda x: userdict[x])
for name,group in testrem.groupby('genre_ids'):
    subcolin = np.array([genredict[sub.strip()] for sub in name.split("|") if sub in train_genre])
    if len(subcolin) > 0:
        vsum = ugmat[:,2*subcolin][group.mind,:].sum(axis = 1)
        vcount = ugmat[:,2*subcolin+1][group.mind,:].sum(axis =1)
        gi = group.index
        for i in range(group.shape[0]):
            test.set_value(gi[i],'ugfreq',vsum[i]/vcount[i] if vcount[i] > 0 else 0.5)
            test.set_value(gi[i],'ugcount',vcount[i]/len(subcolin))
    else:
        for ind in group.index:
            test.set_value(ind,'ugcount',0)
            test.set_value(ind,'ugfreq',group.user_freq[ind])
test = test.drop(['mind','gmind','ugsum'],1)

/Users/Dylan/anaconda/envs/3point6/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [56]:
from sklearn.metrics.pairwise import cosine_similarity
rowin = train['msno'].unique()
colin = train['song_id'].unique()
spm = sparse.lil_matrix((len(rowin),len(colin)))
dictuser = dict(zip(rowin,range(len(rowin))))
dictsong = dict(zip(colin,range(len(colin))))
for ind in train.index:
    userp = dictuser[train.msno[ind]]
    songp = dictsong[train.song_id[ind]]
    spm[userp,songp] = (1 if train.target[ind] == 1 else -1)
u_simi_song= cosine_similarity(spm)
spm = sparse.csr_matrix(spm)

In [59]:
train.drop(['usbs', 'usbs_is_estimate', 'uind', 'rating'],1, inplace=True)

In [60]:
members['rtime'] = members['registration_init_time'].apply(
    lambda x : pd.to_datetime(str(x), format='%Y%m%d'))
members['etime'] = members['expiration_date'].apply(
    lambda x : pd.to_datetime(str(x), format='%Y%m%d'))
members['numactivedays'] = (members['etime'] - members['rtime']).dt.days
min_day = members['rtime'].min()
members['day_id_res'] = (members['rtime'] - min_day).dt.days+1
members['day_id_exp'] = (members['etime'] - min_day).dt.days+1
members['r_year'] = members['rtime'].dt.year
members['r_month'] = members['rtime'].dt.month
members['r_day'] = members['rtime'].dt.day
members['e_year'] = members['etime'].dt.year
members['e_month'] = members['etime'].dt.month
members['e_day'] = members['etime'].dt.day

In [61]:
members['is_male'] = members.gender == 'male'
members['is_female'] = members.gender == 'female'
members['age_mtrue'] = members.bd.apply(lambda x: 1 if x > 10 and x < 80 else 0)
weighted_age = round(members.bd[members.age_mtrue == 1].mean())
members['age_clean'] = members.bd.apply(lambda x: x if x >10 and x < 80 else weighted_age)

In [62]:
train = train.merge(members,on = 'msno', how = 'left')
test = test.merge(members,on = 'msno', how = 'left')
valid = valid.merge(members,on = 'msno', how = 'left')

In [63]:
def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return 1900 + int(isrc[5:7])
        else:
            return 2000 + int(isrc[5:7])
    else:
        return np.nan

In [65]:
songs_extra = pd.read_csv('../1001_proj/song_extra_info.csv/song_extra_info.csv')
songs_extra['song_year'] = songs_extra['isrc'].apply(isrc_to_year)
songs_extra.drop(['isrc', 'name'], axis = 1, inplace = True)

train = train.merge(songs_extra, on='song_id', how='left')
test = test.merge(songs_extra, on='song_id', how='left')
valid = valid.merge(songs_extra, on='song_id', how='left')

In [70]:
for col in train.columns:
    if train[col].dtype == object:
        train[col] = train[col].astype('category')
        test[col] = test[col].astype('category')
        valid[col] = valid[col].astype('category')

In [74]:
import datetime
import time
train.rtime = train.rtime.apply(lambda x : time.mktime(x.timetuple()))
train.etime = train.etime.apply(lambda x : time.mktime(x.timetuple()))

In [75]:
valid.rtime = valid.rtime.apply(lambda x : time.mktime(x.timetuple()))
valid.etime = valid.etime.apply(lambda x : time.mktime(x.timetuple()))

In [76]:
X_train = train.drop(['target'], axis=1)
y_train = train['target'].values

X_valid = valid.drop(['target'], axis=1)
y_valid = valid['target'].values

In [77]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train, y_train)
lgb_val = lgb.Dataset(X_valid, y_valid)

In [79]:
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'learning_rate': 0.2 ,
        'verbose': 0,
        'num_leaves': 100,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 1,
        'max_bin': 256,
        'num_rounds': 10,
        'metric' : 'auc'
    }
lgbm_model = lgb.train(params, train_set = lgb_train, valid_sets = lgb_val, verbose_eval=5)

/Users/Dylan/anaconda/envs/3point6/lib/python3.6/site-packages/lightgbm/engine.py:98: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[5]	valid_0's auc: 0.668207
[10]	valid_0's auc: 0.672774


In [82]:
test.rtime = test.rtime.apply(lambda x : time.mktime(x.timetuple()))
test.etime = test.etime.apply(lambda x : time.mktime(x.timetuple()))
X_test = test.drop(['id'], axis=1)
ids = test['id'].values
predictions = lgbm_model.predict(X_test)

# Writing output to file
subm = pd.DataFrame()
subm['id'] = ids
subm['target'] = predictions
subm.to_csv('lgbm_submission.csv.gz', compression = 'gzip', index=False, float_format = '%.5f')
